In [1]:
from IPython.display import display

import pandas as pd
import warnings
from model_drift import settings, helpers
from model_drift.data.utils import nested2series
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns
from model_drift.drift.tabular import TabularDriftCalculator
from model_drift.drift.numeric import KSDriftCalculator, BasicDriftCalculator
from model_drift.drift.categorical import ChiSqDriftCalculator
from model_drift.drift.collection import DriftCollectionCalculator
from model_drift.drift.performance import AUROCCalculator

from model_drift.drift.sampler import Sampler
from model_drift.data.padchest import PadChest
import plotly.graph_objects as go

warnings.filterwarnings("ignore")


In [2]:
jsonl_file = str(settings.TOP_DIR.joinpath("results", 'vae', 'padchest-trained', 'preds.jsonl'))
vae_df = helpers.jsonl_files2dataframe(jsonl_file)
vae_df = pd.concat(
    [
        vae_df,
        pd.DataFrame(vae_df['mu'].values.tolist(), columns=[f"mu.{c}" for c in range(128)])
    ],
    axis=1
)
vae_df.head()

100%|██████████| 160819/160819 [00:18<00:00, 8655.80it/s]


,index,mu,logvar,error,mu.0,mu.1,mu.2,mu.3,mu.4,mu.5,...,mu.118,mu.119,mu.120,mu.121,mu.122,mu.123,mu.124,mu.125,mu.126,mu.127
0,20536686640136348236148679891455886468_k6ga29.png,"[0.004903476685285568, 0.00975932739675045, -0...","[0.00420594634488225, -0.032170265913009644, -...",0.007225,0.004903,0.009759,-0.840139,-0.016406,0.012096,0.021054,...,0.008214,-0.124864,-0.011458,-0.035856,-0.005253,-0.525054,-0.012718,-0.015304,1.603692,1.275674
1,113855343774216031107737439268243531979_3k951n...,"[-0.039043813943862915, -0.007190241944044828,...","[-0.013759884051978588, -0.012844430282711983,...",0.006071,-0.039044,-0.007190,-1.345544,0.016048,0.037869,-0.036479,...,0.042117,0.080279,0.014332,-0.000268,-0.019087,0.168344,-0.046811,-0.030200,-0.740812,-1.276945
2,313723174271082992847610802266403640553-4_hhi4...,"[-0.03004823438823223, -0.05634206160902977, 0...","[-0.010940825566649437, 0.017154257744550705, ...",0.013905,-0.030048,-0.056342,0.508030,-0.041828,-0.014656,0.010803,...,0.050155,0.336182,0.111974,0.046037,0.092280,-1.214425,0.014846,0.073163,0.226776,0.311751
3,105529804483623054726294337265160703666_6zn76a...,"[0.001704810420051217, -0.014846593141555786, ...","[-0.0063932850025594234, 0.010232711210846901,...",0.004213,0.001705,-0.014847,-0.050661,-0.013582,-0.015157,-0.007484,...,0.000933,0.042541,-0.017803,0.016669,-0.019601,-1.454505,-0.013419,0.012802,0.410283,0.671507
4,102185472428957491598043403159908631419_zb8kco...,"[-0.03474627062678337, -0.021671226248145103, ...","[-0.022024642676115036, -0.014861042611300945,...",0.009243,-0.034746,-0.021671,-1.811071,0.097347,-0.022849,-0.029674,...,-0.045857,-0.070322,0.004691,0.022646,-0.038137,0.053273,-0.028840,0.028752,-1.016176,-1.146801


In [3]:
from model_drift.data.padchest import LABEL_MAP
label_cols = list(LABEL_MAP)
jsonl_file = str(settings.TOP_DIR.joinpath("results", 'classifier', 'finetuned', "preds.jsonl"))
scores_df = helpers.jsonl_files2dataframe(jsonl_file)
scores_df = pd.concat(
    [
        scores_df,
        pd.DataFrame(scores_df['activation'].values.tolist(), columns=[f"activation.{c}" for c in label_cols])
    ],
    axis=1)
scores_df.head()


100%|██████████| 160819/160819 [00:03<00:00, 46974.64it/s]


,index,score,activation,label,activation.Atelectasis,activation.Cardiomegaly,activation.Consolidation,activation.Edema,activation.Lesion,activation.No Finding,activation.Opacity,activation.Pleural Abnormalities,activation.Pleural Effusion,activation.Pneumonia
0,20536686640136348236148679891455886468_k6ga29.png,"[-2.9898717403411865, 0.021230269223451614, -5...","[0.04788553714752197, 0.5053073763847351, 0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",0.047886,0.505307,0.003428,0.000492,0.031599,0.040083,0.046906,0.223817,0.010098,0.010910
1,113855343774216031107737439268243531979_3k951n...,"[-4.271155834197998, -3.2341253757476807, -7.5...","[0.013773280195891857, 0.03790152817964554, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.013773,0.037902,0.000529,0.000097,0.022773,0.269458,0.025679,0.003461,0.000689,0.003102
2,313723174271082992847610802266403640553-4_hhi4...,"[-1.9546648263931274, -4.485781669616699, -1.7...","[0.12404559552669525, 0.011142521165311337, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.124046,0.011143,0.148217,0.000012,0.022932,0.003598,0.751884,0.655488,0.479314,0.211316
3,105529804483623054726294337265160703666_6zn76a...,"[-2.843719244003296, -2.7252206802368164, -5.9...","[0.055006884038448334, 0.06150144338607788, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.055007,0.061501,0.002591,0.000115,0.016477,0.370590,0.017562,0.013063,0.001621,0.005887
4,102185472428957491598043403159908631419_zb8kco...,"[-2.794471502304077, -4.529303073883057, -5.34...","[0.05762365832924843, 0.01067304890602827, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.057624,0.010673,0.004754,0.000176,0.021283,0.097500,0.051552,0.037033,0.019453,0.026915


In [4]:
# Load padchest CSV
pc = PadChest(settings.PADCHEST_FILENAME)
pc.prepare()


pc.merge(vae_df, left_on="ImageID", right_on="index", how='inner')
pc.merge(scores_df, left_on="ImageID", right_on="index", how='inner')


train, val, test = pc.split(settings.PADCHEST_SPLIT_DATES, studydate_index=True)


In [5]:
pd.concat(
    {
        "all": pc.df["StudyDate"].describe(datetime_is_numeric=True),
        "train": train.df["StudyDate"].describe(datetime_is_numeric=True),
        "val": val.df["StudyDate"].describe(datetime_is_numeric=True),
        "test": test.df["StudyDate"].describe(datetime_is_numeric=True),
    },
    axis=1,
)


,all,train,val,test
count,160819,91726,22176,46917
mean,2012-09-14 20:54:45.910246912,2011-01-06 03:16:23.616423168,2013-06-16 00:14:59.999999744,2015-08-29 00:15:02.359485696
min,2007-05-03 00:00:00,2007-05-03 00:00:00,2013-01-01 00:00:00,2014-01-01 00:00:00
25%,2010-10-27 00:00:00,2010-01-19 00:00:00,2013-03-10 00:00:00,2014-08-08 00:00:00
50%,2012-06-18 00:00:00,2011-01-18 00:00:00,2013-06-04 00:00:00,2015-06-09 00:00:00
75%,2014-05-28 00:00:00,2012-01-11 00:00:00,2013-09-25 00:00:00,2016-09-13 00:00:00
max,2017-11-17 00:00:00,2012-12-28 00:00:00,2013-12-31 00:00:00,2017-11-17 00:00:00


In [6]:
FLOAT = KSDriftCalculator
CAT = ChiSqDriftCalculator

cols = {
    'age': FLOAT,
    'Projection': CAT,
    "PatientSex_DICOM": CAT,
    "ViewPosition_DICOM": CAT,
    "Modality_DICOM": CAT,
    "Manufacturer_DICOM": CAT,
    "PhotometricInterpretation_DICOM": CAT,
    "PixelRepresentation_DICOM": CAT,
    "PixelAspectRatio_DICOM": CAT,
    "SpatialResolution_DICOM": CAT,
    "BitsStored_DICOM": CAT,
    "WindowCenter_DICOM": FLOAT,
    "WindowWidth_DICOM": FLOAT,
    "Rows_DICOM": FLOAT,
    "Columns_DICOM": FLOAT,
    "XRayTubeCurrent_DICOM": CAT,
    "Exposure_DICOM": CAT,
    "ExposureInuAs_DICOM": FLOAT,
    "RelativeXRayExposure_DICOM": FLOAT,
}

cols.update({c:FLOAT for c in list(pc.df) if c.startswith("mu.") and 'all' not in c})
cols.update({c:FLOAT for c in list(pc.df) if c.startswith("activation.") and 'all' not in c})
cols[("score", "label")] = AUROCCalculator


In [7]:
window = "30D"
stride = "D"
ref_frontal_only = True
target_frontal_only = True
min_periods = 20

replacement = True
sample_size = 500
n_samples = 10


sampler = Sampler(sample_size, replacement=replacement)


In [8]:
refdf = val.df.copy()
if ref_frontal_only:
    refdf = refdf.query("Frontal")


print(len(refdf), len(val.df))

dwc = TabularDriftCalculator(refdf)

for c, kls in cols.items():
    dwc.add_drift_stat(c, kls)
dwc.prepare()

target_df = pc.df.set_index('StudyDate')

if target_frontal_only:
    target_df = target_df.query("Frontal")

print(len(target_df), len(pc.df.set_index('StudyDate')))


15267 22176
110610 160819


In [9]:
output = dwc.rolling_window_predict(target_df,
                                    sampler=sampler, n_samples=n_samples,
                                    stride=stride, window=window, min_periods=min_periods,
                                    n_jobs=1,
                                    )


2007-05-03 - 2017-11-17 window: 30D, stride: D: 100%|██████████| 3852/3852 [3:57:48<00:00,  3.70s/it, 2017-11-17]


In [10]:
fname = settings.TOP_DIR.joinpath(
    "results", "drift_csvs", f"combined_s{stride}-w{window}-min{min_periods}_frontalonly-ref{ref_frontal_only}-target{target_frontal_only}_Samp-ss{sample_size}-n{n_samples}-repl{replacement}.csv")
print(fname)
output.to_csv(fname)


D:\Code\MLOpsDay2\MedImaging-ModelDriftMonitoring\results\drift_csvs\combined_sD-w30D-min20_frontalonly-refTrue-targetTrue_Samp-ss500-n10-replTrue.csv


In [11]:
output.head()

BitsStored_DICOM                                               \
                       chi2                                                
              critical_diff                     critical_value             
                       mean       obs       std           mean       obs   
2009-01-02        15.197152 -1.142930  6.462779       2.705543  2.705543   
2009-01-03        20.373776 -0.800635  5.879211       2.705543  2.705543   
2009-01-04         0.948156 -2.404752  3.498373       2.705543  2.705543   
2009-01-05         4.030977 -1.461291  3.452606       2.705543  2.705543   
2009-01-06        -1.362254 -2.582056  1.131888       2.705543  2.705543   

                                                              ...  \
                                                              ...   
                           distance                      dof  ...   
                     std       mean       obs       std mean  ...   
2009-01-02  4.681111e-16  17.902696  1.562613  6.462779  1.0  ...   
2009-01-03  4.681111e-16  23.079320  1.904909  5.879211  1.0  ...   
2009-01-04  4.681111e-16   3.653700  0.300791  3.498373  1.0  ...   
2009-01-05  4.681111e-16   6.736521  1.244253  3.452606  1.0  ...   
2009-01-06  4.681111e-16   1.343290  0.123488  1.131888  1.0  ...   

                    mu.99                                              \
                       ks                                               
           critical_value  distance                              pval   
                      std      mean       obs       std          mean   
2009-01-02   7.314236e-18  0.210800  0.211434  0.018250  9.883299e-16   
2009-01-03   7.314236e-18  0.212358  0.206469  0.019040  2.100020e-16   
2009-01-04   7.314236e-18  0.196679  0.192267  0.018306  6.167626e-13   
2009-01-05   7.314236e-18  0.127757  0.126183  0.025308  2.616853e-05   
2009-01-06   7.314236e-18  0.112125  0.113299  0.012043  6.589075e-05   

                                       score                      
                                       label                      
                                       auroc                      
                 obs           std      mean       obs       std  
2009-01-02  0.014758  2.652894e-15  0.904917  0.907737  0.006239  
2009-01-03  0.013121  6.170327e-16  0.908164  0.910047  0.006002  
2009-01-04  0.016304  1.938340e-12  0.907333  0.908202  0.003908  
2009-01-05  0.016873  5.640828e-05  0.919550  0.920855  0.005982  
2009-01-06  0.031096  1.137372e-04  0.922205  0.920836  0.007751  

[5 rows x 1914 columns]